```
Name: Andrea Fox
Date: May 29, 2021
Course: DSC650 - T301 Big Data
Assignment: Assignment 11
```

## Assignment 11
```
Using section 8.1 in Deep Learning with Python as a guide, implement an LSTM text generator. Train the model on the Enron corpus or a text source of your choice. Save the model and generate 20 examples to the results directory of dsc650/assignments/assignment11/.
```

In [42]:
#load libraries
import os
from pathlib import Path
import keras
import numpy as np
import tensorflow as tf
from keras import layers
import random
import sys

In [43]:
#set directories and create results folder
current_dir = Path(os.getcwd()).absolute()
results_dir = current_dir.joinpath('results')
results_dir.mkdir(parents=True, exist_ok=True)

In [44]:
#load in data. Used gutenberg.org and Pride and Prejudice as my text https://gutenberg.org/files/1342/1342-0.txt
path = keras.utils.get_file(
    '1342-0.txt',
    origin='https://gutenberg.org/files/1342')
text = open(path, encoding = 'utf-8').read().lower()
print('Corpus length:', len(text))

Corpus length: 1809


In [45]:
#extract partially-overlapping sequences of length maxlen, one-hot encode them and pack them in a 3D numpy array
# Length of extracted character sequences
maxlen = 60

# We sample a new sequence every `step` characters
step = 3

# This holds our extracted sequences
sentences = []

# This holds the targets (the follow-up characters)
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

# List of unique characters in the corpus
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
# Dictionary mapping unique characters to their index in `chars`
char_indices = dict((char, chars.index(char)) for char in chars)

# Next, one-hot encode the characters into binary arrays.
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 583
Unique characters: 50
Vectorization...


In [46]:
#building the network for a single LSTM layer with Dense classifier and softmax over all characters
model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

#since targets are one-hot encoded, use categorical_crossentropy as the loss to train the model
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

#save model
model.save('LSTM_assignment11_FoxAndrea.h5')

In [47]:
#samping function
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [48]:
#loop to train and generate text
for epoch in range(1, 21):
    print('epoch', epoch)
    # Fit the model for 1 epoch on the available training data
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    # Select a text seed at random
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')

    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

        # We generate 400 characters
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]
            
            #used Robert's code here to save
            results = 'LSTM_Example-Epoch_' + str(epoch) + '_Temp_' + str(temperature) + '.txt'
            with open(results_dir.joinpath(results), 'w') as file:
                file.writelines(generated_text)

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

epoch 1
5/5 [==============================] - 2s 125ms/step - loss: 3.9074
--- Generating with seed: " </td><td align="right">  - </td><td>&nbsp;</td></tr>
   <tr"
------ temperature: 0.2
 </td><td align="right">  - </td><td>&nbsp;</td></tr>
   <tr< i< < <<<<  <<<   </ <   h<<< << <  "<<  <<<<<< < h  l <<  h<<  <n<h<<" <<<<<   <<<"<  << <""   <<<   <<<<<<<n<<h h<<<t<<<  <<<<<<< <  << h<<     << < hhh< h <  <"<<<     <<<<<"h << < h <<<h<    <   <<< < <<<"<<<<  <h<h<<<"       <<<<<<<<<hh<<  < < <  << <    h h  < <"< <<<<<<<h   <<<< <<<<  <  <<h< <  <  <<<<< <<<<h   <<<<<<<< t<< h < <<<<<<< <<<  << < <   <<h< "h<" <h  <   h   < " <<  < h <<<< 
------ temperature: 0.5
<<<<<< <<<  << < <   <<h< "h<" <h  <   h   < " <<  < h <<<< <"< h l"<" < >h<nhiht<<<<">f<hi3" "t>h"r<<oa<a h/t<t n=  s <r<"" cnh ninhias"chhn<"r"<h<<nh/<<tr<t=< <<d"  < <l<"f </>< <fni<d<   < dvhd  <<<  /<h<e<i hii  at<m<<   "<nnh<<dh  nt< < <s-t<<<r" tfi<l<<"i2" h<"h<  df<<hrnh/"<  <- < l2ha " <<e-<<"h hhnhh<toi l <hn  ti <

```
Looking at what printed out in the epochs didn't make a ton of sense. I wonder if it has anything to do with using the website I did. Not sure if others had similar issues, but I used the same website as many others just different text.
```